In [1]:
import pandas as pd
import numpy as np
import ast
import json
from operator import add 
from operator import truediv
import re
# import swifter
from tqdm import tqdm
import time

In [2]:
tech = pd.read_csv("Dataset/Technology Summary Matching Project - technologies.csv")
# summary = pd.read_csv("Dataset/10000_profiles_summary.csv")
# summary = pd.read_csv("Dataset/Test-NewV3 - Testing-Sheet(23_02) - Sheet1.csv")
summary = pd.read_csv("Dataset/1lakh_profiles.csv")
# summary = pd.read_csv("Dataset/10000_profiles_summary.csv")
summary_copy = summary.copy()
tech_copy = tech.copy()

In [3]:
tech.head()

,title
0,Lattice
1,WorkflowMax
2,InGenius
3,DealHub
4,Ebsta


In [4]:
summary.head()

,id,birth_date,birth_date_fuzzy,certifications,education,emails,experience,industries,inferred_salary,inferred_years_experience,interests,locations,names,phone_numbers,primary,profiles,skills,summaries,version_status
0,NoD-ZqKG-uZ08YLoOLgc3g_0000,NaN,NaN,[],"[{""end_date"":""2010"",""majors"":[""nursing""],""scho...",[],"[{""end_date"":""2012-08"",""last_updated"":""null"",""...","[{""is_primary"":true,""name"":""mental health care""}]","85,000-100,000",9.0,[],"[{""continent"":""north america"",""country"":""unite...","[{""is_primary"":true,""name"":""josie sullivan"",""l...",[],"{work_emails=[], personal_emails=[], function=...","[{""aliases"":[],""is_primary"":true,""ids"":[""27506...","[{""name"":""patient advocacy""}, {""name"":""patient...",[],"{""contains"":[],""previous_version"":""9"",""current..."
1,GrCNjsdb-mjQWk3ENPU1AA_0000,NaN,NaN,[],[],[],"[{""end_date"":""null"",""last_updated"":""2020-03-01...","[{""is_primary"":true,""name"":""dairy""}]",NaN,NaN,[],"[{""continent"":""oceania"",""country"":""new zealand...","[{""is_primary"":true,""name"":""keith neylon"",""las...",[],"{work_emails=[], personal_emails=[], function=...","[{""aliases"":[],""is_primary"":true,""ids"":[""10048...","[{""name"":""new business development""}, {""name"":...","[Owner, Blue River Dairy Ltd]","{""contains"":[],""previous_version"":""9"",""current..."
2,XOYPYP6cCikO1i4eSHW3gg_0000,NaN,NaN,[],[],[],"[{""end_date"":""null"",""last_updated"":""2018-12-01...","[{""is_primary"":true,""name"":""non-profit organiz...","45,000-55,000",NaN,[],"[{""continent"":""asia"",""country"":""singapore"",""zi...","[{""is_primary"":true,""name"":""nelly tjondro"",""la...",[],"{work_emails=[], personal_emails=[], function=...","[{""aliases"":[],""is_primary"":true,""ids"":[],""cle...","[{""name"":""cold calling""}, {""name"":""advertising...",[Account Executive at Association for people w...,"{""contains"":[],""previous_version"":""9"",""current..."
3,HEx2xma56Um38KRbybgL4g_0000,NaN,NaN,[],"[{""end_date"":""2007"",""majors"":[],""school"":{""web...",[],"[{""end_date"":""null"",""last_updated"":""2019-12-01...","[{""is_primary"":true,""name"":""construction""}]","55,000-70,000",10.0,[],"[{""continent"":""europe"",""country"":""united kingd...","[{""is_primary"":true,""name"":""daniel hunston"",""l...",[],"{work_emails=[], personal_emails=[], function=...","[{""aliases"":[],""is_primary"":true,""ids"":[""65224...","[{""name"":""project management""}, {""name"":""const...",[Competent in the use of specialist design sof...,"{""contains"":[],""previous_version"":""9"",""current..."
4,PX3Fy29WfbtsnlWiOvqvXA_0000,NaN,NaN,[],"[{""end_date"":""null"",""majors"":[],""school"":{""web...","[{""address"":""briang@innovationado.com"",""sha256...","[{""end_date"":""2017-03"",""last_updated"":""null"",""...","[{""is_primary"":false,""name"":""primary/secondary...","150,000-250,000",14.0,[],"[{""continent"":""north america"",""country"":""unite...","[{""is_primary"":true,""name"":""brian gleason"",""la...",[],"{work_emails=[], personal_emails=[], function=...","[{""aliases"":[],""is_primary"":true,""ids"":[""54693...","[{""name"":""leadership""}, {""name"":""team building...","[innovation expert helping executives, compani...","{""contains"":[""YPoUANpyCZKPLAx3mnnaBw_0000""],""p..."


In [5]:
# # print("data types summaries",type(summary["summaries"][0]))

# # converting string of list data that is being dumped from sourcehere to list properly to iterate easily later on
# for i in range(0, len(summary)):
#     demo = summary["summaries"][i]
#     res = demo.strip('][')
#     resl = []
#     resl.append(res)
#     summary["summaries"][i] = resl

In [6]:
# taking 500 entries from sheet
summary_copy = summary_copy.iloc[0:100] 

In [7]:
tech_list = []
summary_list = []
id_list = []

In [8]:
def KMP_String(pattern, text):
    a = len(text)
    b = len(pattern)

    prefix_arr = get_prefix_arr(pattern, b)

  
    initial_point = []

    m = 0
    n = 0

  
    while m != a:
       
        if text[m] == pattern[n]:
            m += 1
            n += 1
      
        else:
            n = prefix_arr[n-1]

       
        if n == b:
            initial_point.append(m-n)
            n = prefix_arr[n-1]
        elif n == 0:
            m += 1
            
    return initial_point



def get_prefix_arr(pattern, b):
    prefix_arr = [0] * b
    n = 0
    m = 1

    while m != b:
        if pattern[m] == pattern[n]:
            n += 1
            prefix_arr[m] = n
            m += 1
        elif n != 0:
                n = prefix_arr[n-1]
        else:
            prefix_arr[m] = 0
            m += 1

    return prefix_arr

In [9]:
# print("data types summaries",type(summary["summaries"][0]))

# converting string of list data that is being dumped from sourcehere to list properly to iterate easily later on
# for i in range(0, len(summary_copy)):
#     demo = summary_copy["summaries"][i]
#     res = demo.strip('][')
#     resl = []
#     resl.append(res)
#     summary_copy["summaries"][i] = resl

for i in range(0, len(summary_copy)-1):
    print(i)
    if summary_copy["summaries"][i] is not np.nan:
        demo = summary_copy["summaries"][i]
        res = demo.strip('][')
        resl = []
        resl.append(res)
        summary_copy["summaries"][i] = resl
        
    else:
        summary_copy["summaries"][i] = '[]'
        demo = summary_copy["summaries"][i]
        res = demo.strip('][')
        resl = []
        resl.append(res)
        summary_copy["summaries"][i] = resl


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


<ipython-input-9-1a2ce9e9369c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_copy["summaries"][i] = resl


In [10]:
start = time.time()

for i in range(0, len(summary_copy)-1):
    print(i)
    string = summary_copy.loc[i, "summaries"][0]
    string = string.lower()
    
    for j in range(0, len(tech)-1):
        pat = tech.loc[j,"title"]
        pat = pat.lower()
        
        
        initial_index = KMP_String(pat, string)
        if len(initial_index)!=0:
                
            tech_list.append(pat)    
            summary_list.append(string)
            id_list.append(summary_copy.loc[i,"id"])
            
end = time.time() - start
print("\n TImE TAken", end)


# #             for k in range(0, len(initial_index)-2):

# #                 if (initial_index[k]==0) and (string[initial_index[k]+len(pat)-1+1].isalpha()==True):
# #                     print("C1")
# #                     initial_index.pop(k)
# #                     match_dict.update({i:[initial_index, len(pat)]})
# #                     print("Dictionary C1", match_dict)
                    
                    

# #                 elif (string[initial_index[k]-1].isalpha()==True) or (string[initial_index[k]+len(pat)-1+1].isalpha()==True):
# #                     print("C2")
# #                     initial_index.pop(k)
# #                     match_dict.update({i:[initial_index, len(pat)]})
# #                     print("Dictionary C2", match_dict)

# #                 else:
# #                     pass

# #             tech_list.append(pat)    
# #             summary_list.append(string)
# #             id_list.append(summary.loc[i,"id"])
                
# #         else:
# #             pass

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98

 TImE TAken 264.7095549106598


In [11]:
tech_list_copy = tech_list
id_list_copy = id_list
summary_list_copy = summary_list

In [12]:
def getDuplicatesWithInfo(listOfElems):
    ''' Get duplicate element in a list along with thier indices in list
     and frequency count'''
    dictOfElems = dict()
    index = 0
    # Iterate over each element in list and keep track of index
    for elem in listOfElems:
        # If element exists in dict then keep its index in list & increment its frequency
        if elem in dictOfElems:
            dictOfElems[elem][0] += 1
            dictOfElems[elem][1].append(index)
        else:
            # Add a new entry in dictionary 
            dictOfElems[elem] = [1, [index]]
        index += 1    
 
    dictOfElems = { key:value for key, value in dictOfElems.items() if value[0] > 1}
    return dictOfElems

In [13]:
# List of strings
listOfElems = id_list
dictOfElems = getDuplicatesWithInfo(listOfElems)

In [14]:
# tech_list_demo = tech_list

In [15]:
type(listOfElems)

list

In [16]:
# for key, value in dictOfElems.items():
#         print('Element = ', key , ' :: Repeated Count = ', value[0] , ' :: Index Positions =  ', value[1])

In [17]:
df = pd.DataFrame(list(zip(id_list, summary_list, tech_list)), 
               columns =['id', 'summaries', "Technology"])

In [18]:
df.head()

,id,summaries,Technology
0,GrCNjsdb-mjQWk3ENPU1AA_0000,"owner, blue river dairy ltd",air
1,GrCNjsdb-mjQWk3ENPU1AA_0000,"owner, blue river dairy ltd",blue
2,GrCNjsdb-mjQWk3ENPU1AA_0000,"owner, blue river dairy ltd",airy
3,GrCNjsdb-mjQWk3ENPU1AA_0000,"owner, blue river dairy ltd",ada
4,GrCNjsdb-mjQWk3ENPU1AA_0000,"owner, blue river dairy ltd",iri


In [19]:
# df.to_csv("Dataset/Output/op_string/string_match_copy_Save17k.csv", index = False)

In [20]:
for key, value in dictOfElems.items():
#     print("******************")
#     print("\n ID is..", key )
    a = df["Technology"][value[1][0]] #this needs to be changed
    b = []
    b.insert(0,a)
#     print("List at first index", b)
    for i in range(0, len(value[1])-1):
        b.append(df["Technology"][value[1][i+1]])
        df["Technology"][value[1][0]] = b
        
    for i in range(0, len(value[1])-1):
        df = df.drop(index = [value[1][i+1]])
#         print('Element = ', key , ' :: Repeated Count = ', value[0] , ' :: Index Positions =  ', value[1])

In [21]:
df.reset_index(inplace = True, drop = True)

In [22]:
# Applying Regex to filter out Invalid Words

def get_exact_match(txt, tech):
    try:
        patt = '|'.join(['\\b'+elem+'\\b' for elem in tech])
        matched_patt = re.findall(patt,txt)
    except Exception as e:
        print(e)
        matched_patt = []
    return matched_patt

In [23]:
# new_tech = []
# new_summary = []
# new_id = []

In [24]:
# for ind in df.index:
    
#     for j in range(0, len(df)):
#         txt = df["summaries"][i]
#         tech_txt = df["Technology"][j]
#         match = get_exact_match(txt, tech_txt)
        
#         new_tech.append(match)
#         new_summary.append(txt)
#         new_id.append(df["id"][i])

In [25]:
df['exact_matched_patt'] = df.apply(lambda x: get_exact_match(x["summaries"], x["Technology"]), axis = 1)

In [26]:
df.head(-1)

,id,summaries,Technology,exact_matched_patt
0,GrCNjsdb-mjQWk3ENPU1AA_0000,"owner, blue river dairy ltd","[air, blue, airy, ada, iri, ve]",[blue]
1,XOYPYP6cCikO1i4eSHW3gg_0000,account executive at association for people wi...,"[c, ion, sass, ve, h]",[]
2,HEx2xma56Um38KRbybgL4g_0000,competent in the use of specialist design soft...,"[tanda, compete, alis, bric, ink, meet, thin, ...",[]
3,PX3Fy29WfbtsnlWiOvqvXA_0000,"innovation expert helping executives, companie...","[tanda, xperience, vision, industry, rise, rep...","[industry, driven, built, vision]"
4,fgtMhfVKi-MqcB8LPQB0hQ_0000,"proactivo, orientado a resultados, trabajo en ...","[alis, trab, c, activo, iri, tact, h, quip, nt...",[]
...,...,...,...,...
68,FUv4LI1BjO6GYGB-L8NIyw_0000,executive at lic of india,"[c, ve]",[]
69,W7b37g6ShgW-eb9CJJjUHg_0000,experienced head of laboratory with a demonstr...,"[xperience, industry, lisa, disco, disc, ora, ...","[industry, good]"
70,CEs9SI-B784CB3Bnjf5Q1Q_0000,marketing executive at lets go maldives pvt lt...,"[ark, ets, alis, found, c, iri, ve, go]",[go]
71,BtwzGmijkRGDoV8NtwDhKQ_0000,successful current student with an outstanding...,"[xperience, rain, due, and co, continu, manage...","[due, driven, manager, manager, manager]"


In [27]:
# df.to_csv("Dataset/Output/op_string/KMP_Latest-Regex_17_k_profiles_latest.csv", index = False)

In [28]:
# sorting
# df_cm = pd.read_csv("Dataset/Output/op_string/KMP_Latest-Regex_17_k_profiles_latest.csv")

In [29]:
# Python code to sort a list without 
# creating another list Use of sort() 
def Sorting(lst): 
	lst.sort(key=len, reverse = True) 
	return lst 

In [30]:
# for i in range(0, len(df)-1):
#     print(i)
    
#     if df["exact_matched_patt"][i] == '[]':
        
#         res = ast.literal_eval(df["exact_matched_patt"][i])
#         df["exact_matched_patt"][i] = res
        
#     else:
#         x = []
#         x.insert(0, df["exact_matched_patt"][i])
#         df["exact_matched_patt"][i] = x

In [31]:
for ind in df.index:
    df["exact_matched_patt"][ind] = Sorting(df["exact_matched_patt"][ind])

In [32]:
count = []
id_elem = []
elem = []

In [33]:
for ind in df.index:
    d = df["exact_matched_patt"][ind]
    for j in range(0, len(d)-1):
        search_elem = df["exact_matched_patt"][ind][j]
        i = d.count(search_elem)
        count.append(i)
        elem.append(search_elem)
        id_elem.append(df["id"])

In [34]:
df_final_v2 = pd.DataFrame(list(zip(elem, count)), 
               columns =['Technology', "Count"])

In [35]:
df_final_v2.head()

,Technology,Count
0,industry,1
1,driven,1
2,vision,1
3,industry,1
4,industry,1


In [36]:
# Count vs Technology Sheet
save = df_final_v2.groupby(["Technology"]).sum()
save.reset_index(inplace = True)

In [37]:
save.head()

,Technology,Count
0,accept,1
1,accurate,1
2,care,2
3,create,2
4,driven,2


In [38]:
save.to_csv("Tech_Count_17k_groupby.csv", index = False)